# Comparison

Compare my algorithms to DPCP, DPCP-IRLS, RPCA, and WPCA.

Here we gradually add outliers and noise to the dataset and plot the results

In [1]:
import numpy as np
from scipy.linalg import svd, eigh
import time


import sys
sys.path.append('../')
from scripts import flag_dimensionality_reduction as fdr


In [2]:
# WPCA-L2 Code
#from https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8119535&casa_token=RO14HRIjf5UAAAAA:FXvg29pDDWh7AT6-VEj0ERFlHY4wM6nYUejcziYqt8y3yTTGcoQauH8Owo8a0riXa9Wc4BAogg&tag=1
def WPCA_L2(X, k, init_seed = 123, eps_wt = 1e-8 , eps: float = 1e-10,  max_iter: int = 1000):

    n,p = X.shape

    U,_,_ = np.linalg.svd(X)
    W = U[:,:k]

    dJ = 1
    J = 0
    i=1

    while i < max_iter and dJ > eps*J:

        W0 = W.copy()
        J = np.sum(np.linalg.norm(X.T - X.T @ W0 @ W0.T, axis = 1))
    
        D = np.zeros((p,p))
        for i in range(p):
            x = X[:,[i]]
            wt = np.linalg.norm(x - W0 @ W0.T @ x)
            wt = np.max([wt,eps_wt])
            D[i,i] = 1/wt
        
        C = X @ D @ X.T

        _, Wall = eigh(C)

        W = Wall[:,-k:]

        Bj = np.sum(np.linalg.norm(X.T - X.T @ W@ W.T, axis = 1))
        

        dJ = J - Bj
        i+=1


    return W


In [3]:
#DPCP-IRLS Code

def dpcp_irls(X: np.array, k: int, eps: float = 1e-10, delta: float = 1e-8, max_iter: int = 100):
    
    n_pts = X.shape[1]

    U,_,_ = np.linalg.svd(X)
    B0 = U[:,-k:]

    J = 0
    dJ = 1
    i = 0
    while i < max_iter and dJ > eps*J:
            
        J = np.sum(np.linalg.norm(X.T @ B0, axis = 1))
        
        wts = [np.sqrt(1/np.max([delta, np.linalg.norm(B0.T @ X[:,[i]])])) for i in range(n_pts)]
        W = np.diag(np.hstack(wts))

        U,_,_ = np.linalg.svd(X @ W)

        B = U[:,-k:]

        Bj = np.sum(np.linalg.norm(X.T @ B, axis = 1))

        dJ = J - Bj

        B0 = B
        i+= 1
    
    return B0

In [4]:
# RPCA Code
# translated from https://github.com/Markopoulos-Research/Efficient-L1-Norm-Principal-Component-Analysis-via-Bit-Flipping

def rpca_l1(X, K, num_init, print_flag):
    # Parameters
    toler =10e-8

    # Get the dimensions of the matrix.
    D = X.shape[0]	# Row dimension.
    N = X.shape[1]	# Column dimension.

    # Initialize the matrix with the SVD.
    _, S_x, V_x = svd(X , full_matrices = False)	# Hint: The singular values are in vector form.
    if D < N:
        V_x = V_x.T
    
    Y = np.diag(S_x)@V_x.T # X_t is Y = S*V'
    # Initialize the required matrices and vectors.
    Bprop = np.ones((N,K),dtype=float)
    nucnormmax = 0
    iterations = np.zeros((1,num_init),dtype=float)
    # For each initialization do.
    for ll in range(0, num_init):

        start_time = time.time()	# Start measuring execution time.

        z = X.T @ np.random.randn(D,1)	# Random initialized vector.
        if ll<1:    # In the first initialization, initialize the B matrix to sign of the product of the first
                    # right singular vector of the input matrix with an all-ones matrix.
            z = V_x[:,0]
            z = z.reshape(N,1)
        v = z@np.ones((1,K), dtype=float)
        v = np.random.randn(N,K)
        B = np.sign(v)	# Get a binary vector containing the signs of the elements of v.

        iter_ = 0
        L = list(range(N * K))

        while True:
            iter_ = iter_ + 1
            # Calculate all the possible binary vectors and all possible bit flips.

            a = np.zeros((N,K)) # nuclear norm of when the (m,l)th bit of B is flipped
            
            nucnorm = np.linalg.norm(Y@B, 'nuc')
            
            for x in L:
                l = x//N
                m = x-N*l
                elK = np.zeros(K)
                elK[l] = 1
                a[m,l] = np.linalg.norm(Y@B - 2*B[m,l]*(Y[:,m,None]@ [elK]), 'nuc')
            nucnorm_flip = np.max(a) # Choose the largest nuclear norm of YB

            n,k = np.unravel_index(np.nanargmax(a, axis=None), a.shape) # Pick the best bit flip

            if nucnorm_flip > nucnorm: # If the best bit flip increases the nuclear norm of YB, then flip the bit
                B[n,k] = -B[n,k]
                L.remove(k*N+n) # No longer flip that (n,k) bit
            elif nucnorm_flip <= nucnorm + toler and len(L)<N*K: # Else, but there has been bit-flips, reset bit-flipping process
                L = list(range(N*K))
            else:
                break

        # Calculate the final subspace.
        U, _, V = svd(X@B, full_matrices=False)
        Utemp = U[:,0:K]
        Vtemp = V[:,0:K]
        Q = Utemp@Vtemp.T
        
        nucnorm = sum(sum(abs(Q.T@X)))
        
        # Find the maximum nuclear norm across all initializations.
        if nucnorm > nucnormmax:
            nucnormmax = nucnorm
            Bprop = B
            Qprop = Q
            vmax = nucnorm
        iterations[0,ll] = iter_

    end_time = time.time()	# End of execution timestamp.
    timelapse = (end_time - start_time)	# Calculate the time elapsed.

    convergence_iter = np.mean(iterations, dtype=float) # Calculate the mean iterations per initialization.
    
    if print_flag:
        print("------------------------------------------")
        print("Avg. iterations/initialization: ", (convergence_iter))
        print("Time elapsed (sec):", (timelapse))
        print("Metric value:", vmax)
        print("------------------------------------------")

    return Qprop, Bprop, vmax

Start with comparing on a small dataset

$$
\{ \mathbf{x}_j \}_{j=1}^{100} \in \mathbb{R}^5 \:\:\:\: \mathbf{x}_{i,j} \sim \mathcal{U}[0,1)
$$

In [9]:
n = 5
p= 100
np.random.seed(189)
X = np.random.rand(n,p)

column_means = np.mean(X.T, axis=0)
Xcenter = X.T - column_means
Xcenter = Xcenter.T

Xunit = np.vstack([x/np.linalg.norm(x) for x in X.T]).T

In [10]:
pca_type = 'rpca'
fl_type = [1,2]
eyes = fdr.construct_eyes(fl_type)

print(f'testing {pca_type}{fl_type}')
print()

start = time.time()
Wmine, errs, cauchs = fdr.flag_robust_pca(Xcenter, fl_type, pca_type, verbose = False, return_all = True, max_iters = 200, init = 'rand')
print('fRPCA(1,2)')
print(f'elapsed time: {time.time() - start}')
print(f'objective value: {fdr.objective_val(Xcenter, Wmine, eyes, pca_type)}')
print()


start = time.time()
Wbase, _,_ = rpca_l1(Xcenter, 2, 200, False)
print('L1-RPCA')
print(f'elapsed time: {time.time() - start}')
print(f'objective value: {fdr.objective_val(Xcenter, Wbase, eyes, pca_type)}')




testing rpca[1, 2]

fRPCA(1,2)
elapsed time: 0.1902148723602295
objective value: 54.65498478146488

L1-RPCA
elapsed time: 70.65553784370422
objective value: 54.69002134791764


In [11]:
pca_type = 'wpca'
fl_type = [2]
eyes = fdr.construct_eyes(fl_type)

print(f'testing {pca_type}{fl_type}')
print()

start = time.time()
Wmine, errs, cauchs = fdr.flag_robust_pca(Xcenter, fl_type, pca_type, verbose = False, return_all = True, max_iters = 200, init = 'rand')
print('fWPCA(2)')
print(f'elapsed time: {time.time() - start}')
print(f'objective value: {fdr.objective_val(Xcenter, Wmine, eyes, pca_type)}')
print()


start = time.time()
Wbase = WPCA_L2(Xcenter, fl_type[-1])
print('L2-WPCA')
print(f'elapsed time: {time.time() - start}')
print(f'objective value: {fdr.objective_val(Xcenter, Wbase, eyes, pca_type)}')


testing wpca[2]

fWPCA(2)
elapsed time: 0.45084309577941895
objective value: 42.917794060592385

L2-WPCA
elapsed time: 0.2361278533935547
objective value: 42.8859158409493


In [12]:
pca_type = 'dpcp'
fl_type = [2]
eyes = fdr.construct_eyes(fl_type)

print(f'testing {pca_type}{fl_type}')
print()

start = time.time()
Wmine, errs, cauchs = fdr.flag_robust_pca(Xunit, fl_type, pca_type, verbose = False, return_all = True, max_iters = 200, init = 'rand')
print('fDPCP(2)')
print(f'elapsed time: {time.time() - start}')
print(f'objective value: {fdr.objective_val(Xcenter, Wmine, eyes, pca_type)}')

start = time.time()
Wbase = dpcp_irls(Xunit, fl_type[-1])
print('L2-DPCP')
print(f'elapsed time: {time.time() - start}')
print(f'objective value: {fdr.objective_val(Xcenter, Wbase, eyes, pca_type)}')


testing dpcp[2]

fDPCP(2)
elapsed time: 0.3977639675140381
objective value: 34.659257598447454
L2-DPCP
elapsed time: 0.2617058753967285
objective value: 34.83425152370253
